In [10]:
import os
import re
import cv2
import imutils
import sys
import pytesseract
import image
import numpy as np
import docx
import tensorflow as tf
import skimage
import keras
import PIL
import time
import pyautogui
import pandas as pd
import keras

from matplotlib import pyplot as plt
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from autocorrect import spell
from nltk.corpus import words
from docx import Document
from docx.shared import Pt
from docx.enum.section import WD_ORIENT
from keras.preprocessing.image import load_img ,img_to_array ,array_to_img
from skimage import data, io, filters, color
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from PIL import Image
from numpy import zeros, newaxis
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D



def Read_TrainDATA_Facial_Expression():    

########### #######         ##       # ###      ##      #######         ##   ###########   ##                                      
     #      ##     ##      ####      # ## #     ##      ##     ##      ####       #       ####                        
     #      ##     ##     ##  ##     # ##  #    ##      ##      ##    ##  ##      #      ##  ##                              
     #      #######      ##    ##    # ##   #   ##      ##      ##   ##    ##     #     ##    ##                            
     #      ##   ##     ##########   # ##    #  ##      ##      ##  ##########    #    ##########                                
     #      ##    ##   ##        ##  # ##     # ##      ##     ##  ##        ##   #   ##        ##                              
     #      ##     ## ##          ## # ##      ###      #######   ##          ##  #  ##          ##                       
                                                           
    # Assign Arrays
    ImageArray = []
    EmotionList = []
    
    # Read Data from Train Data csv file
    ReadCSV = pd.read_csv(Train_Facial_Expression)
    ImageList = ReadCSV['pixels'].tolist()
    Emotion_No = ReadCSV['emotion'].tolist()
    
    # Convert Numbers to Emotions
    # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
    for Counter, Value in enumerate(Emotion_No):
        if Value   == 0:
            EmotionList.append('Angry')
        elif Value == 1:
            EmotionList.append('Disgust')
        elif Value == 2:
            EmotionList.append('Fear')
        elif Value == 3:
            EmotionList.append('Happy')            
        elif Value == 4:
            EmotionList.append('Sad')            
        elif Value == 5:
            EmotionList.append('Surprise')            
        elif Value == 6:
            EmotionList.append('Neutral')     
            
    for I in ImageList:
        Each_Image = [  int(pixel) for pixel in I.split(' ')  ]         
        ImageArray.append(np.asarray(Each_Image).reshape(48,48,1)) 
    
    
##    # CSV Summary
##    ReadCSV.info()
       
##    # Display Train Data Facial Expression on Screen
##    for J in range(10):
##        plt.figure(figsize=(1.5,1.5))
##        plt.title(EmotionList[J], fontsize=15)
##        plt.imshow(ImageArray[J])
        
        
    return ImageArray, Emotion_No, EmotionList 
        

    


        
        
        

        
def Machine_Learning_Model(ImageArray):  

###       ###  #######  #######    ######## #                                                                             
####     #### ##     ## ##     ##  #        #                                                         
##  #   #  ## ##     ## ##      ## #        #                                           
##   # #   ## ##     ## ##      ## ######   #                                                        
##    #    ## ##     ## ##      ## #        #                                                         
##         ## ##     ## ##     ##  #        #                                                              
##         ##  #######  #######    ######## ########                                                                                                    

                                
    ImageArray = np.array(ImageArray)   
##    ImageArray = ImageArray / 255.0   
    x_train, x_test, y_train, y_test = train_test_split(ImageArray, Emotion_No, test_size=0.2, shuffle=True)
    x_train, x_test = x_train / 255.0, x_test / 255.0 
##    print(x_train.shape) # Print the Shape/Dimension of train data as (28709, 48, 48, 1)=(samples, rows, columns, channels)
                         # Note: x_train is IMAGE & y_train is LABEL
                         # Note: x_test  is IMAGE & y_test  is LABEL
   
    
##    # Display Train Data on Screen
##    for J in range(10):
##        plt.figure(figsize=(1.5,1.5))
##        plt.title(y_train[J], fontsize=15)
##        plt.imshow(x_train[J])     
    

##    # Neural Network Model 
##    model = tf.keras.models.Sequential([
##      tf.keras.layers.Flatten(input_shape=(48,48)),      # Input Layer
##      tf.keras.layers.Dense(128, activation=tf.nn.relu),  # Hidden Layer
##      tf.keras.layers.Dense(7, activation=tf.nn.softmax)  # Output Layer
##    ])


################################################# CNN #################################################
# When Filter is 32, there will have 32 different Feature Maps
# The more Feature Maps there are, the picture becomes more abstract
# Max pooling is used to find the outliers, this is when network sees the feature

    # Convolutional Neural Network (CNN)
    model = Sequential()  
    model.add(Conv2D(64, kernel_size=(5, 5), activation=tf.nn.relu, input_shape=(48,48,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(Dropout(0.1))
    
    model.add(Conv2D(128, kernel_size=(5, 5), activation=tf.nn.relu))
    model.add(MaxPooling2D(pool_size=(2, 2)))    
#    model.add(Dropout(0.1))   
    
################################################# CNN #################################################    
    

    model.add(Flatten())
##    model.add(Dense(256, activation=tf.nn.swish))
    model.add(Dense(256, activation=tf.nn.relu))
    model.add(Dense(7, activation=tf.nn.softmax))

    # Compile Model
##    model.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   
    model.fit(x_train, y_train, epochs=1)  
       
    # Model Score
    score = model.evaluate(x_test, y_test, verbose=100)
    
    # Save / Load Model Weights
    model.save_weights(os.path.join(Current_Dir,'Model_Weights\DigitWeights_Trial.h5'))
##    model.load_weights(os.path.join(Current_Dir,'Model_Weights\DigitWeights_42.4.h5'))
    
##    # Model Summary
##    model.summary()
    
    return score, model        










def Facial_Expression_Prediction(image_file):      

#######  #######    ######## #######    #   ####### ###########                                                 
##     # ##     ##  #        ##     ##  #  ##            #                                 
##     # ##     ##  #        ##      ## # ##             #                      
#######  #######    ######   ##      ## # ##             #                         
##       ##    ##   #        ##      ## # ##             #                                  
##       ##     ##  #        ##     ##  #  ##            #                              
##       ##      ## ######## #######    #   #######      #                                       
        
    
        # Convert image to Numbers
        image_file = np.array(image_file)
 
        # shrink image to 48x48 size. This size must fit Train Data size 
        image_file = skimage.transform.resize(image_file, [48,48], mode='reflect')
        image_file = np.reshape(image_file,(1,48,48,1))         # 4 dimensional image for Tensorflow to process
        
        # Do Prediction
        Prediction = model.predict(image_file)
        
        # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
        if Prediction.argmax()   == 0:
            Emotion = 'Angry'
        elif Prediction.argmax() == 1:
            Emotion = 'Disgust'
        elif Prediction.argmax() == 2:
            Emotion = 'Fear'
        elif Prediction.argmax() == 3:
            Emotion = 'Happy'            
        elif Prediction.argmax() == 4:
            Emotion = 'Sad'            
        elif Prediction.argmax() == 5:
            Emotion = 'Surprise'            
        elif Prediction.argmax() == 6:
            Emotion = 'Neutral'        
            
        return Emotion, Prediction

            
            
            
            
            
            
            
            
            
            
            
def Facial_Expression_Recognition(Test_Facial, Frontal_Face_Detection_XML, Profile_Face_Detection_XML, Eyes_Detection_XML):

#######    ########  ######   #######   ######   ##      # ####### ####### #######  #######  ##      #                     
##     ##  #        ##     # ##     ## ##        ###     #    #       #       #    ##     ## ###     #
##     ##  #        ##       ##     ## ##        #  #    #    #       #       #    ##     ## #  #    #     
#######    ######   ##       ##     ## ##   #### #   #   #    #       #       #    ##     ## #   #   #     
##    ##   #        ##       ##     ## ##     ## #    #  #    #       #       #    ##     ## #    #  #           
##     ##  #        ##     # ##     ## ##     ## #     ###    #       #       #    ##     ## #     ###            
##      ## ########  ######   #######   #######  #      ## #######    #    #######  #######  #      ##                      


    # Angle of face detection
    Frontal_face_cascade = cv2.CascadeClassifier(Frontal_Face_Detection_XML)
    Profile_face_cascade = cv2.CascadeClassifier(Profile_Face_Detection_XML)
    eye_cascade  = cv2.CascadeClassifier(Eyes_Detection_XML)


    VideoFrame = cv2.VideoCapture(0)
    VideoFrame.set(4,640) # set Width
    VideoFrame.set(5,480) # set Height    
    while(True):
        ret, image_file_for_display = VideoFrame.read()
        image_file_for_display = cv2.flip(image_file_for_display, 1)
        image_file = cv2.cvtColor(image_file_for_display, cv2.COLOR_BGR2GRAY)
        faces = Frontal_face_cascade.detectMultiScale(image_file_for_display, scaleFactor=1.25, minNeighbors=5, \
                minSize=(10, 10))
        
        for (x,y,w,h) in faces:  
            Emotion, Prediction = Facial_Expression_Prediction(image_file[y:y+h,x:x+w])
            cv2.putText(image_file_for_display, str(Emotion) + " " + str(round(Prediction[0][Prediction.argmax()]*100,1)) + "%", \
                        org=(x,y-10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.0,                       \
                        color=(0, 0, 255), thickness=1                    
                       )            
            cv2.rectangle(image_file_for_display,(x,y),(x+w,y+h),(0,0,255),2) 
            
        cv2.imshow('Colour', image_file_for_display)
##        cv2.imshow('Non-Colour', image_file)
    
        k = cv2.waitKey(30) & 0xff
        if k == 27: # press 'ESC' to quit
            break
    VideoFrame.release()
    cv2.destroyAllWindows()


        
        
        
        

        
        
        
def Duration_Calculation():        

########### ###### ###       ### #######              
    ##        ##   ####     #### #                 
    ##        ##   ##  #   #  ## #                  
    ##        ##   ##   # #   ## #####                 
    ##        ##   ##    #    ## #                   
    ##        ##   ##         ## #    
    ##      ###### ##         ## ########            
                     
    EndTime = time.time()
    EndDateTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    # Calculate script Runtime
    Duration = EndTime - StartTime
    hours = Duration//3600
    Duration = Duration - 3600*hours
    minutes = Duration//60
    seconds = Duration - 60*minutes
    
    print("Start:", StartDateTime,                                      '\t    Loss:', round(score[0], 3)   )
    print("End  :", EndDateTime,                                        '\tAccuracy:', round(score[1], 3)   )
    print('Runtime: %d Hrs %d Min %d Sec' %(hours,minutes,seconds)) 
    
    
    


        
        
   

        
###     ###      ###      ## ###    ##           
####   ####     ## ##     ## ####   ##                 
## ## ## ##    ##   ##    ## ## ##  ##              
##  ###  ##   #########   ## ##  ## ##                  
##       ##  ##       ##  ## ##   ####                         
##       ## ##         ## ## ##    ###                       

# Script process Start Time
StartTime = time.time()
StartDateTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

Current_Dir = os.getcwd()              # Get this script current directory


                        ########## LOCATION OF INPUTS, FOLDERS & IMAGES ##########
# Train Data Location    
Train_Facial_Expression = os.path.join(Current_Dir,'Train_DATA\Train_DATA_Facial_Expressions.csv')  

# Test Data Location
Test_Facial = [os.path.join(Current_Dir,'DATA',X) for X in os.listdir(os.path.join(Current_Dir,'DATA'))]   

# Frontal Face Detection
Frontal_Face_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_frontalface_default.xml')

# Profile Face Detection
Profile_Face_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_profileface.xml')

# Eyes Detection
Eyes_Detection_XML = os.path.join(Current_Dir,'Face_Detection\haarcascade_eye.xml')
    

                        ########## FUNCTIONS ##########
ImageArray, Emotion_No, EmotionList = Read_TrainDATA_Facial_Expression()
score, model = Machine_Learning_Model(ImageArray)
Facial_Expression_Recognition(Test_Facial, Frontal_Face_Detection_XML, Profile_Face_Detection_XML, Eyes_Detection_XML)
Duration_Calculation()





Epoch 1/1
28709/28709 [==============================] - 475s 17ms/step - loss: 1.6183 - acc: 0.3613
Start: 2019-11-07 15:17:20 	    Loss: 1.464
End  : 2019-11-07 15:51:16 	Accuracy: 0.435
Runtime: 0 Hrs 33 Min 56 Sec
